In [12]:
from azureml.core import Dataset
from azureml.core import Workspace
import datetime

ws = Workspace.from_config()
dataset = Dataset.get_by_name(ws, name='BTC-Bitstamp-raw')

df = dataset.to_pandas_dataframe()
df = df.rename(columns={"Timestamp": "dateid", "Close": "price","Volume_(BTC)":"volume"})


df

,dateid,Open,High,Low,price,volume,Volume_(Currency),Weighted_Price
0,1325317920,4.39,4.39,4.39,4.39,0.455581,2.000000,4.390000
1,1325317980,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1325318040,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1325318100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1325318160,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
4857372,1617148560,58714.31,58714.31,58686.00,58686.00,1.384487,81259.372187,58692.753339
4857373,1617148620,58683.97,58693.43,58683.97,58685.81,7.294848,428158.146640,58693.226508
4857374,1617148680,58693.43,58723.84,58693.43,58723.84,1.705682,100117.070370,58696.198496
4857375,1617148740,58742.18,58770.38,58742.18,58760.59,0.720415,42332.958633,58761.866202


In [13]:
dateparse = lambda x: datetime.datetime.utcfromtimestamp(float(x))
df['date'] = [dateparse(x) for x in df['dateid']]
df = df.set_index('date')
del df['dateid']
df['ask'] = df['price']*1.01
df['bid'] = df['price']*0.99
df = df.reindex(columns=['price','ask','bid','volume','High','Low'])
df

,price,ask,bid,volume,High,Low
date,,,,,,
2011-12-31 07:52:00,4.39,4.4339,4.3461,0.455581,4.39,4.39
2011-12-31 07:53:00,NaN,NaN,NaN,NaN,NaN,NaN
2011-12-31 07:54:00,NaN,NaN,NaN,NaN,NaN,NaN
2011-12-31 07:55:00,NaN,NaN,NaN,NaN,NaN,NaN
2011-12-31 07:56:00,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2021-03-30 23:56:00,58686.00,59272.8600,58099.1400,1.384487,58714.31,58686.00
2021-03-30 23:57:00,58685.81,59272.6681,58098.9519,7.294848,58693.43,58683.97
2021-03-30 23:58:00,58723.84,59311.0784,58136.6016,1.705682,58723.84,58693.43


In [14]:
df = df.dropna()
 #df['prev'] = df['price'].rolling(2).apply(lambda x: (x[1]/x[0])-1 ,raw=True)
    
#df['ma100']=df['price']/df['price'].rolling(10).mean()-1
df['ma10001']=df['price']/df['price'].rolling('15min').mean()-1
#df['ma10002']=df['price']/df['price'].rolling('60s').max()-1
#df['ma10003']=df['price']/df['price'].rolling('60s').min()-1

df['ma1000x1']=df['price']/df['price'].rolling('1h').mean()-1
df['ma1000x2']=df['price']/df['High'].rolling('1h').max()-1
df['ma1000x3']=df['price']/df['Low'].rolling('1h').min()-1
df['ma1000x4'] = df['ma1000x1'] / df['price'].rolling('1h').std()

df['ma1000y1']=df['price']/df['price'].rolling('10h').mean()-1
    #df['ma1000y2']=df['price']/df['price'].rolling('10h').max()-1
    #df['ma1000y3']=df['price']/df['price'].rolling('10h').min()-1

df['ma100001']=df['price']/df['price'].rolling('1D').mean()-1
    #df['ma100002']=df['price']/df['price'].rolling('1D').max()-1
    #df['ma100003']=df['price']/df['price'].rolling('1D').min()-1
df['ma100004'] = df['ma100001'] / df['price'].rolling('1D').std()
df['ma1']=df['price']/df['price'].rolling('10D').mean()-1
    #df['ma2']=df['price']/df['price'].rolling('10D').max()-1
    #df['ma3']=df['price']/df['price'].rolling('10D').min()-1
df['ma4'] = df['ma1'] / df['price'].rolling('10D').std()
df['mafff1']=df['price']/df['price'].rolling('20D').mean()-1
df['mafff2']=df['price']/df['High'].rolling('20D').max()-1
df['mafff3']=df['price']/df['Low'].rolling('20D').min()-1
df['mafff4'] = df['mafff1'] / df['price'].rolling('20D').std()
df = df.dropna()

    
del df['High']
del df['Low']

df

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

,price,ask,bid,volume,ma10001,ma1000x1,ma1000x2,ma1000x3,ma1000x4,ma1000y1,ma100001,ma100004,ma1,ma4,mafff1,mafff2,mafff3,mafff4
date,,,,,,,,,,,,,,,,,,
2011-12-31 17:00:00,4.58,4.6258,4.5342,9.000000,0.001093,0.001093,0.000000,0.017778,0.154559,0.021751,0.021751,0.203496,0.021751,0.203496,0.021751,0.000000,0.043280,0.203496
2012-01-03 14:54:00,5.26,5.3126,5.2074,29.319392,0.000000,0.011538,0.000000,0.023346,0.135982,0.003817,0.015444,0.109206,0.090258,0.265182,0.090258,-0.011278,0.198178,0.265182
2012-01-03 15:32:00,5.29,5.3429,5.2371,29.302457,0.000000,0.002844,0.000000,0.005703,0.134049,0.007139,0.016917,0.128171,0.087731,0.249776,0.087731,-0.005639,0.205011,0.249776
2012-01-03 17:14:00,5.14,5.1914,5.0886,0.020000,-0.014382,-0.014382,-0.028355,0.000000,-0.135591,-0.019084,-0.012623,-0.108531,0.046082,0.132170,0.046082,-0.033835,0.170843,0.132170
2012-01-03 17:26:00,5.29,5.3429,5.2371,11.000000,0.014382,0.009542,0.000000,0.029183,0.110181,0.008168,0.014139,0.126540,0.071139,0.203399,0.071139,-0.005639,0.205011,0.203399
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-30 23:56:00,58686.00,59272.8600,58099.1400,1.384487,0.000311,0.000310,-0.001371,0.002030,0.000006,-0.001480,0.005809,0.000009,0.056279,0.000028,0.036386,-0.050109,0.165330,0.000017
2021-03-30 23:57:00,58685.81,59272.6681,58098.9519,7.294848,0.000229,0.000313,-0.001374,0.002027,0.000007,-0.001477,0.005793,0.000009,0.056275,0.000028,0.036380,-0.050112,0.165326,0.000017
2021-03-30 23:58:00,58723.84,59311.0784,58136.6016,1.705682,0.000736,0.000956,-0.000727,0.002676,0.000020,-0.000824,0.006432,0.000010,0.056959,0.000029,0.037050,-0.049497,0.166081,0.000017
